# Zeitzonen exportieren

In [43]:
from pathlib import Path
import geopandas as gpd
import json
import os
import consts

## Create 15° Lines

In [44]:
features = []

for i in range(0, 24):

    lon = (i * 15) - 180 + 7.5

    features.append({
        'type': 'Feature',
        'properties': {
            'timezone': i,
        },
        'geometry': {
            'type': 'LineString',
            'coordinates': [
                [lon, -90],
                [lon, 90],
            ]
        }
    })


json.dump({
    'type': 'FeatureCollection',
    'features': features
}, open(consts.PATH_GEOJSON_TIMEZONES, 'w', encoding='utf-8'), ensure_ascii=False)

## Add Timezones to Country shapes

In [49]:
gdf_countries_raw = gpd.read_file(Path('../data/countries-europe.gpkg'))

In [50]:
gdf_countries = gdf_countries_raw.copy()

# Set Timezone
gdf_countries['timezone_is'] = 1
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['PT', 'GB', 'IE', 'IS', 'IM', 'FO']), 'timezone_is'] = 0
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['FI', 'EE', 'LT', 'LV', 'RO', 'BG', 'GR', 'MD', 'UA']), 'timezone_is'] = 2
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['RU', 'BY', 'TK']), 'timezone_is'] = 3

# Timezone should
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['IS', 'IE', 'PT']), 'timezone_should'] = -1
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['ES', 'GB', 'FR', 'BE', 'NL', 'LU', 'AD', 'JE', 'GG', 'IM', 'FO']), 'timezone_should'] = 0
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin([
    'DE', 'CH', 'IT', 'AT', 'PL', 'CZ', 'SI', 'HR', 'BA', 'ME', 'AL', 'MK', 'XK', 'RS', 'HU', 'SK', 'NO', 'SE', 'DK',
     'MC', 'LI', 'MC', 'SM', 'MT'
]), 'timezone_should'] = 1
gdf_countries.loc[gdf_countries['ISO_A2_EH'].isin(['FI', 'EE', 'LT', 'LV', 'UA', 'RO', 'BG', 'GR', 'MD', 'BY', 'AX']), 'timezone_should'] = 2

# Drop Countries
gdf_countries = gdf_countries[~gdf_countries['ISO_A2_EH'].isin(['RU', 'TK'])]

gdf_countries = gdf_countries[['geometry', 'ISO_A2_EH', 'timezone_is', 'timezone_should']]

gdf_countries.to_file(consts.PATH_GEOJSON_COUNTRIES)

## Merge everything

In [55]:
# Run Tippecanoe
layers = [
    consts.PATH_GEOJSON_EXPORT, # Not needed anymore, remove
    consts.PATH_GEOJSON_COUNTRIES,
    consts.PATH_GEOJSON_TIMEZONES,
    consts.PATH_WEBEXPORT / 'sunrise-current.geojson',
    consts.PATH_WEBEXPORT / 'sunrise-summer.geojson',
    consts.PATH_WEBEXPORT / 'sunrise-winter.geojson',
    consts.PATH_WEBEXPORT / 'sunset-current.geojson',
    consts.PATH_WEBEXPORT / 'sunset-summer.geojson',
    consts.PATH_WEBEXPORT / 'sunset-winter.geojson'
]

layers_str = ' '.join([str(layer) for layer in layers])
# Run Tippecanoe with additional parameters to preserve the shape of hexagons
os.system(f"""tippecanoe -zg -o {consts.PATH_WEBEXPORT / 'zeitumstellung.mbtiles'} {layers_str} --force --extend-zooms-if-still-dropping --no-feature-limit --no-tile-size-limit""")


For layer 0, using name "grid"
For layer 1, using name "countries"
For layer 2, using name "timezonelines"
For layer 3, using name "sunrisecurrent"
For layer 4, using name "sunrisesummer"
For layer 5, using name "sunrisewinter"
For layer 6, using name "sunsetcurrent"
For layer 7, using name "sunsetsummer"
For layer 8, using name "sunsetwinter"
43803 features, 10753538 bytes of geometry, 700126 bytes of string pool
Choosing a maxzoom of -z1 for features typically 162148 feet (49423 meters) apart, and at least 54232 feet (16530 meters) apart
Choosing a maxzoom of -z4 for resolution of about 25006 feet (7622 meters) within features
  99.9%  4/0/5  


0

In [ ]:
consts.PATH_GEOJSON_EXPORT

PosixPath('../export/web/grid.geojson')